In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb


We will be looking at school specific data from across the United States. Our goal is to see if any schools resemble eachother. Often schools are described as "good", but if you are not familiar with the area you have little idea what that definition of "good" means. We want to explore the school graduation rates, diversity, expulsion rates, and teacher ratios to get an idea of what a "good" school is.

If we are also able to cluster like schools together, you may be able to see how your current school (something you are familiar with) compares to other schools you are looking to move into.

## Data sources:

2015-2016 graduation rates: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 school location, characteristics, staff, and lunch programs: https://nces.ed.gov/ccd/pubschuniv.asp

2015-2016 school achievement: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 suspenion rates: https://ocrdata.ed.gov/DataAnalysisTools/DataSetBuilder?Report=4
for this analysis, we only focused on the greenville district schools, but you could select other areas of interest.
    

In [62]:
#load data
path='/home/natalie/Documents/SchoolAPI/Data_Sources/'
grad_2015=pd.read_csv(path+'graduation_rates/2015_school_graduation.csv', parse_dates=True)


In [63]:
#school specific data
school_lunch=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_lunches_ccd_sch_033_1516.csv',  parse_dates=True)
#school_char=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_characteristics_ccd_sch_129_1516_w_2a_011717.csv', parse_dates=True)
school_dir =pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_directory_ccd_sch_029_1516_w_2a_011717.csv',  parse_dates=True)
school_member=pd.read_csv(path+ 'CCD_comoncoredata_nonfiscal/schools_membership_ccd_sch_052_1516_w_2a_011717.csv' ,  parse_dates=True)
school_staff = pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_staff_ccd_sch_059_1516_w_2a_011717.csv')

/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,14,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
#School achievement results for math and reading

math=pd.read_csv(path+'Subject_assesments/school_math-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)
reading= pd.read_csv(path+'Subject_assesments/rla-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)

# percent is the % above the state's "proficient" level
#colums starting with F or M denote breakdowns by gender


In [65]:
#greenville suspensions
suspensions=pd.read_csv(path+'civil_rights/suspensions_greenville_2015.csv')

In [66]:
suspensions.head()
#n = individual counts of students
#otherwise in percentage
#times white students is a multiplier of how much more likely they are to be suspended than a white student

District ID       District State     School_ID                School Name  \
0      4502310  Greenville 01    SC  450231000049        Heritage Elementary   
1      4502310  Greenville 01    SC  450231000114         Oakview Elementary   
2      4502310  Greenville 01    SC  450231000449  Chandler Creek Elementary   
3      4502310  Greenville 01    SC  450231000455             Mauldin Middle   
4      4502310  Greenville 01    SC  450231000459           Riverside Middle   

   AME_n_susp  ASI_n_susp  HI PAC_n_susp  BLA_n_susp  HIS_n_susp  \
0           0           0              0           2           0   
1           0           0              0           6           0   
2           0           2              0          12           6   
3           0           2              0          37           4   
4           0           0              0          20           7   

        ...         HIS_OSS_times_white  TWO_OSS_times_white  \
0       ...                        0.00                 0.00   
1       ...                        0.00                 0.00   
2       ...                        1.84                 0.00   
3       ...                        0.77                 3.62   
4       ...                        1.36                 3.53   

   AME_n_enrollment ASI_n_enrollment HI_PAC_n_enrollment BLA_n_enrollment  \
0                 0                4                   0               61   
1                10               79                   0               91   
2                19                4                   0              214   
3                 4               37                   2              214   
4                 2               76                   2              121   

  HIS_n_enrollment WHI_n_enrollment TWO_n_enrollment TOT_n_enrollment  
0               16              565               25              671  
1               37              697               46              960  
2              190              526               64             1017  
3               82              631               55             1025  
4              109              805               49             1164  

[5 rows x 35 columns]

## Lets pair down the data to only include the variables we are interested in, and then do the cleaning.

### For the math and reading dataste,these variables are:

STNAM = state name

LEANM = education agency name

NCESSCH = unique school ID

SCHNAM = school name

ALL_MTH00NUMVALID_1516  = number of students aggregated across all grades who took test
ALL_MTH00PCTPROF_1516= percentage of all students who are proficient in math across all grades

F_MTH00NUMVALID_1516 = females who took math exam across all grades
F_MTH00PCTPROF_1516 = females proficient in math across all grades

M_MTH00NUMVALID_1516 = males who took math exam across all grades
M_MTH00PCTPROF_1516 = males proficient in math across all grades

MBL_MTHHSNUMVALID_1516 = number of black students in high school who took math examns
MBL_MTHHSPCTPROF_1516 = percent of black students in high school who are proficient in math

MWH_MTHHSNUMVALID_1516 = Number of white students in high school who took math exams
MWH_MTHHSPCTPROF_1516 = percent of white students in high school who are proficient in math

same thing for reading, but MTH is replaced with RLA

In [67]:
#keep only variables listed above

math_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_MTH00NUMVALID_1516', 'ALL_MTH00PCTPROF_1516', 'F_MTH00NUMVALID_1516', 'F_MTH00PCTPROF_1516', 'M_MTH00NUMVALID_1516', 'M_MTH00PCTPROF_1516', 'MBL_MTHHSNUMVALID_1516', 'MBL_MTHHSPCTPROF_1516' , 'MWH_MTHHSNUMVALID_1516', 'MWH_MTHHSPCTPROF_1516']
math=math[math_keep]


reading_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_RLA00NUMVALID_1516', 'ALL_RLA00PCTPROF_1516', 'F_RLA00NUMVALID_1516', 'F_RLA00PCTPROF_1516', 'M_RLA00NUMVALID_1516', 'M_RLA00PCTPROF_1516', 'MBL_RLAHSNUMVALID_1516', 'MBL_RLAHSPCTPROF_1516' , 'MWH_RLAHSNUMVALID_1516', 'MWH_RLAHSPCTPROF_1516']
reading=reading[reading_keep]

math.head(11)

STNAM             LEANM      NCESSCH                        SCHNAM  \
0   ALABAMA  Albertville City  10000500870         Ala Avenue Middle Sch   
1   ALABAMA  Albertville City  10000500871          Albertville High Sch   
2   ALABAMA  Albertville City  10000500879                Evans Elem Sch   
3   ALABAMA  Albertville City  10000500889          Albertville Elem Sch   
4   ALABAMA   Marshall County  10000600193  Kate Duncan Smith DAR Middle   
5   ALABAMA   Marshall County  10000600872                    Asbury Sch   
6   ALABAMA   Marshall County  10000600876        Claysville Jr High Sch   
7   ALABAMA   Marshall County  10000600877              Douglas Elem Sch   
8   ALABAMA   Marshall County  10000600878              Douglas High Sch   
9   ALABAMA   Marshall County  10000600880               Grassy Elem Sch   
10  ALABAMA   Marshall County  10000600883     Kate D Smith DAR High Sch   

    ALL_MTH00NUMVALID_1516 ALL_MTH00PCTPROF_1516 F_MTH00NUMVALID_1516  \
0                      707                    36                  347   
1                      307                    15                  161   
2                      772                    43                  389   
3                      852                    55                  421   
4                      443                    36                  206   
5                      299                    33                  141   
6                       37                 40-49                   20   
7                      494                    55                  235   
8                      148                   6-9                   66   
9                      235                    40                  102   
10                     116                 15-19                   61   

   F_MTH00PCTPROF_1516 M_MTH00NUMVALID_1516 M_MTH00PCTPROF_1516  \
0                   39                  360                  34   
1                15-19                  146               10-14   
2                   46                  383                  40   
3                   54                  431                  57   
4                30-34                  237               35-39   
5                35-39                  158               30-34   
6                40-59                   17               21-39   
7                60-64                  258               45-49   
8                  6-9                   82                 6-9   
9                40-44                  133               40-44   
10               20-24                   55               11-19   

   MBL_MTHHSNUMVALID_1516 MBL_MTHHSPCTPROF_1516 MWH_MTHHSNUMVALID_1516  \
0                     NaN                   NaN                    NaN   
1                       9                  LT50                    177   
2                     NaN                   NaN                    NaN   
3                     NaN                   NaN                    NaN   
4                     NaN                   NaN                    NaN   
5                     NaN                   NaN                     46   
6                     NaN                   NaN                    NaN   
7                     NaN                   NaN                    NaN   
8                       3                    PS                    108   
9                     NaN                   NaN                    NaN   
10                      1                    PS                    112   

   MWH_MTHHSPCTPROF_1516  
0                    NaN  
1                  20-24  
2                    NaN  
3                    NaN  
4                    NaN  
5                   LE10  
6                    NaN  
7                    NaN  
8                    6-9  
9                    NaN  
10                 15-19

### School directory dataset, we are interested in these variables:
    
FIPST = state name
NCESSCH = school ID
LCITY = location city
LZIP = location zip code
WEBSITE = website
G1OFFERED = grade 1 offered
G8OFFERED = grade 8 offered
G12FFERED = grade 12 offered
NOGRADES = no grades offered, if true this school is not operational


In [68]:
dir_keep=['FIPST', 'NCESSCH', 'LCITY', 'LZIP', 'G1OFFERED', 'G8OFFERED', 'G12OFFERED' ,'NOGRADES']
school_dir=school_dir[dir_keep]

### School membership

This is breakdown in grade by race and gender

NCESSCH = school ID
SCH_NAME = school name
GO1 = number of students in grade 1
GO8 = number of students in grade 8
G12 = number of students in grade 12
TOTAL = total students in all grades in school
AM = number of all students in school american indian
AMALM = '' male
AMALF = "" female
AS = number of all students asian
ASALM = "" male
ASALF = "" female
HI = number of all students Hispanic
HIALM
HIALF
BL = number of all students black
BLALM
BLALF
WH = number of all stduents white
WHALM
WHALF
HP = number of all students hawaiian
HPALM
HPALF
TR = number of all students 2 or more races
TRALM
TRALF

AS01M = american indian Male
AS01F = american indian female
HI01M = hispanic Male
HI01F = hispanic female grade1
BL01M = black male grade 1
BL01F = black Female grade1
WH01M = white male grade 1
WH01F = white female grade 1
HP01M = hawaiian/pacific islander male
HP01F = hawaiian/pacific islander Female
TR01M = two or more races male grade 1
TR01F = two or more races female grade 1



In [69]:
school_mem_keep=['NCESSCH', 'SCH_NAME', 'G01', 'G08', 'G12', 'TOTAL', 'AM', 'AMALM', 'AMALF', 'AS', 'ASALM', 'ASALF', 'HI', 'HIALM', 'HIALF', 'BL', 'BLALM', 'BLALF', 'WH', 'WHALM', 'WHALF', 'HP', 'HPALM', 'HPALF', 'TR', 'TRALM', 'TRALF']
school_member=school_member[school_mem_keep]

### School staff dataset, we are interested in these variables:

NCESSCH = school ID
FTE = full time equivalent teachers (value -1 is missing)

In [70]:
school_staff=school_staff[['NCESSCH','FTE']]

### Graduation rate dataset

NCESSCH = school ID

MAM = American Indian/Alaska Native students

MAS = Asian/Pacific Islander

MHI = Hispanic/Latino

MBL = Black

MWH = White

MTR = Two or more races

CWD = disabilities

ECD = Economically disadvanted

LEP = Limited English Proficiency

COHORT = number of students

RATE = percentage in that group out of the entire student population of that school

In [71]:
grad_keep=['NCESSCH', 'ALL_COHORT_1516', 'ALL_RATE_1516', 'MAM_RATE_1516', 'MAS_RATE_1516', 'MBL_RATE_1516', 'MHI_RATE_1516', 'MTR_RATE_1516', 'MWH_RATE_1516', 'CWD_RATE_1516', 'ECD_RATE_1516', 'LEP_RATE_1516']
grad_2015=grad_2015[grad_keep]

### Suspensions dataset

School_ID = school ID
TOT_n_susp = total number of suspensions in that school
TOT_OSS_rate = total number of suspensions in school/ school enrollment

" " OSS_Rate = number of students in this race suspended/ total student of that race enrollment in school

" " OSS_times_white = the rate of suspensions for this race is xx times that of white students

In [72]:
susp_keep=['School_ID', 'TOT_n_susp', 'AME_OSS_times_white', 'ASI_OSS_times_white', 'HI_PAC_OSS_times_white', 'BLA_OSS_times_white', 'HIS_OSS_times_white', 'TWO_OSS_times_white', 'TOT_n_enrollment']

suspensions=suspensions[susp_keep]

In [73]:
suspensions.head()

School_ID  TOT_n_susp  AME_OSS_times_white  ASI_OSS_times_white  \
0  450231000049          25                  0.0                 0.00   
1  450231000114          16                  0.0                 0.00   
2  450231000449          29                  0.0                28.41   
3  450231000455          94                  0.0                 0.83   
4  450231000459          72                  0.0                 0.00   

   HI_PAC_OSS_times_white  BLA_OSS_times_white  HIS_OSS_times_white  \
0                     0.0                 0.79                 0.00   
1                     0.0                 4.52                 0.00   
2                     0.0                 3.24                 1.84   
3                     0.0                 2.71                 0.77   
4                     0.0                 3.55                 1.36   

   TWO_OSS_times_white  TOT_n_enrollment  
0                 0.00               671  
1                 0.00               960  
2                 0.00              1017  
3                 3.62              1025  
4                 3.53              1164

# Data Cleaning

#### 1. the rates of graduation or math/reading competence can be given in values '50-60', or 'GE50' = greater than 50%, or 'PS' which means the data was intentially hidden so you could not focus in on a very small group (e.g., 2 students are hispanic, so any data on these 2 students would wrongly expose their identity). To be conservative, we will select the lower range 50-60, we would select '50' to represent the graduation rate. For values with GE greater than or LE less than, we simply select the value stated, so GE80 becomes '80'.

In [74]:
import re
grad_variables=list(grad_2015)


In [75]:
grad_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 12 columns):
NCESSCH            23090 non-null int64
ALL_COHORT_1516    23090 non-null int64
ALL_RATE_1516      23090 non-null object
MAM_RATE_1516      19434 non-null object
MAS_RATE_1516      20357 non-null object
MBL_RATE_1516      21736 non-null object
MHI_RATE_1516      22034 non-null object
MTR_RATE_1516      20705 non-null object
MWH_RATE_1516      22716 non-null object
CWD_RATE_1516      22656 non-null object
ECD_RATE_1516      22936 non-null object
LEP_RATE_1516      20740 non-null object
dtypes: int64(2), object(10)
memory usage: 2.1+ MB


In [76]:
replace_list=['GE', 'LE', 'LT']

for word in replace_list:
    reading=reading.replace(word, '', regex=True)
    math=math.replace(word, '', regex=True)
    grad_2015=grad_2015.replace(word, '', regex=True) #regex=TRUE checks for partial match, otherwise it would need to match it 100%


In [77]:
grad_2015.head()

NCESSCH  ALL_COHORT_1516 ALL_RATE_1516 MAM_RATE_1516 MAS_RATE_1516  \
0  10000500871              296            92            PS            PS   
1  10000600872               67            95            PS             .   
2  10000600878              153         85-89             .             .   
3  10000600883              120         80-84             .             .   
4  10000601585               94         85-89            PS            PS   

  MBL_RATE_1516 MHI_RATE_1516 MTR_RATE_1516 MWH_RATE_1516 CWD_RATE_1516  \
0            50            95             .         90-94         60-79   
1            PS            80             .            90            50   
2            PS            90             .         85-89            50   
3             .            PS             .         80-84         21-39   
4            PS            PS            PS         85-89            50   

  ECD_RATE_1516 LEP_RATE_1516  
0         80-84            50  
1            90            PS  
2         85-89            PS  
3         60-69             .  
4         80-89             .

In [78]:
na_list=['.', 'PS']

for word in na_list:
    reading=reading.replace(word, np.nan)
    math=math.replace(word, np.nan)
    grad_2015=grad_2015.replace(word, np.nan) #here we want an exact match, so we do not include reges

In [79]:
#removehe the ranges, instead select the lowest value in the range, if rate is 80-85, we only want to keep 80
#grad_2015['ALL_RATE_1516'] = grad_2015['ALL_RATE_1516'].str.split('-').str[0]

r=re.compile(".*RATE")
rate_variables = list(filter(r.match, grad_variables)) # Read Note

for variable in rate_variables:
    grad_2015[variable]= grad_2015[variable].str.split('-').str[0]

r=re.compile(".*PROF")
prof_variables=list(filter(r.match, list(math)))

for variable in prof_variables:
    math[variable]= math[variable].str.split('-').str[0]

In [80]:
#same thing remove dashes in reading variables
r=re.compile(".PROF")
prof_variables=list(filter(r.match, list(reading)))

for variable in prof_variables:
    reading[variable]= reading[variable].str.split('-').str[0]


#### 2. Create class variables in school directory dataset

If a school offers grade 12 and not grade 1 it is considered a High School. If a school offers grade 1 but not grade 8 it is considered elementary. If a school offers grade 8 but not 12 or 1, it is a middle school

In [81]:
#G12OFFERED = YES then this is a high school
#G1OFFERED = YES and G12OFFERED=NO then this is an elementary school
#first replace yes and no with 1 and 0, respectively

school_dir=school_dir.replace(['yes', 'Yes'], 1, regex=True)
#school_dir1=school_dir.replace('Yes', 1)
school_dir=school_dir.replace('No', 0, regex=True)

In [121]:
school_dir['type_school']=np.nan

#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')
      
school_dir['type_school'] = np.where((school_dir['G1OFFERED']==1) & (school_dir['G12OFFERED']==0) , 'elementary',
                np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==1) , 'high_school', 
                    np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==0)& (school_dir['G8OFFERED']==1) , 'middle',  'unknown')))


In [122]:
school_dir[['G1OFFERED', 'G8OFFERED', 'G12OFFERED',  'type_school']].head(10)

G1OFFERED  G8OFFERED  G12OFFERED  type_school
0          0          1           1  high_school
1          0          0           0      unknown
2          0          1           1  high_school
3          0          1           1  high_school
4          0          1           1  high_school
5          0          1           1  high_school
6          0          0           0      unknown
7          0          0           0      unknown
8          0          1           0       middle
9          0          0           0      unknown

In [84]:
#Change all the Cohort variables (those with specific numbers of students to float instead of object)
r=re.compile(".*COHORT")
cohort_variables = list(filter(r.match, grad_variables))

for variable in rate_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')
    
for variable in cohort_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')

In [85]:
grad_2015.describe() #everything we would expect is numeric and can be summarized. These columns can also be used to creat a distance matrix between the different schools.

NCESSCH  ALL_COHORT_1516  ALL_RATE_1516  MAM_RATE_1516  \
count  2.309000e+04      23090.00000   21537.000000    1400.000000   
mean   2.779916e+11        158.25929      76.398106      55.506429   
std    1.595166e+11        179.91495      21.973247      14.389704   
min    1.000050e+10          1.00000       1.000000       6.000000   
25%    1.201980e+11         31.00000      70.000000      50.000000   
50%    2.723535e+11         87.00000      85.000000      50.000000   
75%    4.025388e+11        235.00000      90.000000      60.000000   
max    5.900197e+11       4713.00000      99.000000      95.000000   

       MAS_RATE_1516  MBL_RATE_1516  MHI_RATE_1516  MTR_RATE_1516  \
count    5250.000000    9858.000000   11900.000000    4418.000000   
mean       67.515429      66.321668      65.982353      58.618379   
std        18.621002      20.036679      20.865298      14.619950   
min        10.000000       1.000000       2.000000       5.000000   
25%        50.000000      50.000000      50.000000      50.000000   
50%        60.000000      70.000000      70.000000      50.000000   
75%        80.000000      80.000000      80.000000      80.000000   
max        99.000000      98.000000      99.000000      95.000000   

       MWH_RATE_1516  CWD_RATE_1516  ECD_RATE_1516  LEP_RATE_1516  
count   18344.000000   15049.000000   20125.000000    6279.000000  
mean       76.838203      57.239883      69.643776      56.932155  
std        19.856361      16.321042      20.553885      18.175620  
min         5.000000       5.000000       2.000000       2.000000  
25%        60.000000      50.000000      50.000000      50.000000  
50%        85.000000      50.000000      80.000000      50.000000  
75%        90.000000      70.000000      85.000000      70.000000  
max        99.000000      95.000000      99.000000      95.000000

Clustering algorithims cannot handle missing data, what data is missing?

In [86]:
na_list=grad_2015.isna().sum()
na_list/len(grad_2015) # percentage
#Because many of these race groups are so small the PS identifier was used, and then converted to NA.

NCESSCH            0.000000
ALL_COHORT_1516    0.000000
ALL_RATE_1516      0.067259
MAM_RATE_1516      0.939368
MAS_RATE_1516      0.772629
MBL_RATE_1516      0.573062
MHI_RATE_1516      0.484625
MTR_RATE_1516      0.808662
MWH_RATE_1516      0.205544
CWD_RATE_1516      0.348246
ECD_RATE_1516      0.128411
LEP_RATE_1516      0.728064
dtype: float64

In [114]:
# Fill missing values with mean column values in those columns
grad_2015.fillna(grad_2015.mean(), inplace=True)

In [105]:
reading.fillna(reading.mean(), inplace=True)
math.fillna(math.mean(), inplace=True)

In [88]:
suspensions.fillna(suspensions.mean(), inplace=True)

In [117]:
grad_2015.isna().sum()/len(grad_2015) #all NAS removed

NCESSCH       0.0
ALL_COHORT    0.0
ALL_RATE      0.0
MAM_RATE      0.0
MAS_RATE      0.0
MBL_RATE      0.0
MHI_RATE      0.0
MTR_RATE      0.0
MWH_RATE      0.0
CWD_RATE      0.0
ECD_RATE      0.0
LEP_RATE      0.0
dtype: float64

In [119]:
school_member.isna().sum()

NCESSCH                0
SCH_NAME_n_students    0
G01_n_students         0
G08_n_students         0
G12_n_students         0
TOTAL_n_students       0
AM_n_students          0
AMALM_n_students       0
AMALF_n_students       0
AS_n_students          0
ASALM_n_students       0
ASALF_n_students       0
HI_n_students          0
HIALM_n_students       0
HIALF_n_students       0
BL_n_students          0
BLALM_n_students       0
BLALF_n_students       0
WH_n_students          0
WHALM_n_students       0
WHALF_n_students       0
HP_n_students          0
HPALM_n_students       0
HPALF_n_students       0
TR_n_students          0
TRALM_n_students       0
TRALF_n_students       0
dtype: int64

In [90]:
suspensions.head()

School_ID  TOT_n_susp  AME_OSS_times_white  ASI_OSS_times_white  \
0  450231000049          25                  0.0                 0.00   
1  450231000114          16                  0.0                 0.00   
2  450231000449          29                  0.0                28.41   
3  450231000455          94                  0.0                 0.83   
4  450231000459          72                  0.0                 0.00   

   HI_PAC_OSS_times_white  BLA_OSS_times_white  HIS_OSS_times_white  \
0                     0.0                 0.79                 0.00   
1                     0.0                 4.52                 0.00   
2                     0.0                 3.24                 1.84   
3                     0.0                 2.71                 0.77   
4                     0.0                 3.55                 1.36   

   TWO_OSS_times_white  TOT_n_enrollment  
0                 0.00               671  
1                 0.00               960  
2                 0.00              1017  
3                 3.62              1025  
4                 3.53              1164

## Merge all the data for individual schools together, using 'NCESSCH' as the key id variable.

In [91]:
#before we merge data, lets rename some of the variables so they do not get confused

school_member= school_member.add_suffix('_n_students')
school_member.rename(columns={'NCESSCH_n_students': 'NCESSCH'}, inplace=True)
school_member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99264 entries, 0 to 99263
Data columns (total 27 columns):
NCESSCH                99264 non-null int64
SCH_NAME_n_students    99264 non-null object
G01_n_students         99264 non-null int64
G08_n_students         99264 non-null int64
G12_n_students         99264 non-null int64
TOTAL_n_students       99264 non-null int64
AM_n_students          99264 non-null int64
AMALM_n_students       99264 non-null int64
AMALF_n_students       99264 non-null int64
AS_n_students          99264 non-null int64
ASALM_n_students       99264 non-null int64
ASALF_n_students       99264 non-null int64
HI_n_students          99264 non-null int64
HIALM_n_students       99264 non-null int64
HIALF_n_students       99264 non-null int64
BL_n_students          99264 non-null int64
BLALM_n_students       99264 non-null int64
BLALF_n_students       99264 non-null int64
WH_n_students          99264 non-null int64
WHALM_n_students       99264 non-null int64
WHALF_n_st

In [92]:
#grad_2015 = grad_2015.rename(columns={col: col.split('_')[1] for col in grad_2015.columns})

grad_2015=grad_2015.rename(columns = lambda x : str(x)[:-5])
grad_2015.rename(columns={'NC': 'NCESSCH'}, inplace=True)
grad_2015.head()

NCESSCH  ALL_COHORT  ALL_RATE   MAM_RATE   MAS_RATE   MBL_RATE  \
0  10000500871       296.0      92.0  55.506429  67.515429  50.000000   
1  10000600872        67.0      95.0  55.506429  67.515429  66.321668   
2  10000600878       153.0      85.0  55.506429  67.515429  66.321668   
3  10000600883       120.0      80.0  55.506429  67.515429  66.321668   
4  10000601585        94.0      85.0  55.506429  67.515429  66.321668   

    MHI_RATE   MTR_RATE  MWH_RATE  CWD_RATE  ECD_RATE   LEP_RATE  
0  95.000000  58.618379      90.0      60.0      80.0  50.000000  
1  80.000000  58.618379      90.0      50.0      90.0  56.932155  
2  90.000000  58.618379      85.0      50.0      85.0  56.932155  
3  65.982353  58.618379      80.0      21.0      60.0  56.932155  
4  65.982353  58.618379      85.0      50.0      80.0  56.932155

In [100]:
math2=math
math2=math2.rename(columns=lambda x: str(x) [:-5])

objs=[math.iloc[:,0:3], math2.iloc[:,4:]]

math=pd.concat(objs, axis=1)


STNAM                                 LEANM       NCESSCH  \
0      ALABAMA                      Albertville City   10000500870   
1      ALABAMA                      Albertville City   10000500871   
2      ALABAMA                      Albertville City   10000500879   
3      ALABAMA                      Albertville City   10000500889   
4      ALABAMA                       Marshall County   10000600193   
5      ALABAMA                       Marshall County   10000600872   
6      ALABAMA                       Marshall County   10000600876   
7      ALABAMA                       Marshall County   10000600877   
8      ALABAMA                       Marshall County   10000600878   
9      ALABAMA                       Marshall County   10000600880   
10     ALABAMA                       Marshall County   10000600883   
11     ALABAMA                       Marshall County   10000601434   
12     ALABAMA                       Marshall County   10000601585   
13     ALABAMA                       Marshall County   10000601685   
14     ALABAMA                       Marshall County   10000601812   
15     ALABAMA                       Marshall County   10000602209   
16     ALABAMA                           Hoover City   10000700091   
17     ALABAMA                           Hoover City   10000700248   
18     ALABAMA                           Hoover City   10000700251   
19     ALABAMA                           Hoover City   10000700337   
20     ALABAMA                           Hoover City   10000700342   
21     ALABAMA                           Hoover City   10000701422   
22     ALABAMA                           Hoover City   10000701456   
23     ALABAMA                           Hoover City   10000701483   
24     ALABAMA                           Hoover City   10000701738   
25     ALABAMA                           Hoover City   10000701739   
26     ALABAMA                           Hoover City   10000701740   
27     ALABAMA                           Hoover City   10000701741   
28     ALABAMA                           Hoover City   10000701742   
29     ALABAMA                           Hoover City   10000701743   
...        ...                                   ...           ...   
89379  WYOMING    Fremont County School District #24  560570000323   
89380  WYOMING    Fremont County School District #24  560570000362   
89381  WYOMING    Fremont County School District #24  560570000510   
89382  WYOMING  Sweetwater County School District #2  560576200164   
89383  WYOMING  Sweetwater County School District #2  560576200285   
89384  WYOMING  Sweetwater County School District #2  560576200324   
89385  WYOMING  Sweetwater County School District #2  560576200326   
89386  WYOMING  Sweetwater County School District #2  560576200329   
89387  WYOMING  Sweetwater County School District #2  560576200332   
89388  WYOMING  Sweetwater County School District #2  560576200399   
89389  WYOMING  Sweetwater County School District #2  560576200424   
89390  WYOMING  Sweetwater County School District #2  560576200425   
89391  WYOMING  Sweetwater County School District #2  560576200439   
89392  WYOMING    Washakie County School District #2  560582000393   
89393  WYOMING       Teton County School District #1  560583000289   
89394  WYOMING       Teton County School District #1  560583000335   
89395  WYOMING       Teton County School District #1  560583000336   
89396  WYOMING       Teton County School District #1  560583000337   
89397  WYOMING       Teton County School District #1  560583000339   
89398  WYOMING       Teton County School District #1  560583000340   
89399  WYOMING       Teton County School District #1  560583000341   
89400  WYOMING       Teton County School District #1  560583000512   
89401  WYOMING      Weston County School District #7  560609000342   
89402  WYOMING      Weston County School District #7  560609000363   
89403  WYOMING      Weston County School District #7  560609000401   
89404  WYOMING    W

In [104]:
reading2=reading
reading2=reading2.rename(columns=lambda x: str(x) [:-5])

objs=[reading.iloc[:,0:3], reading2.iloc[:,4:]]
reading=pd.concat(objs, axis=1)
reading.head()

STNAM             LEANM      NCESSCH ALL_RLA00PC F_RLA00NUM F_RLA00PC  \
0  ALABAMA  Albertville City  10000500870          33        339        39   
1  ALABAMA  Albertville City  10000500871          31        162     30-34   
2  ALABAMA  Albertville City  10000500879          30        381        36   
3  ALABAMA  Albertville City  10000500889          36        414        37   
4  ALABAMA   Marshall County  10000600193          42        205     45-49   

  M_RLA00NUM M_RLA00PC MBL_RLAHSNUM MBL_RLAHSPC MWH_RLAHSNUM MWH_RLAHSPC  
0        348        28          NaN         NaN          NaN         NaN  
1        145     25-29            9          50          178       35-39  
2        378        23          NaN         NaN          NaN         NaN  
3        425        34          NaN         NaN          NaN         NaN  
4        237     35-39          NaN         NaN          NaN         NaN

In [106]:
#suspensions.columns[0]
suspensions.rename(columns={'School_ID': 'NCESSCH'}, inplace=True)

In [120]:
school_dir

FIPST       NCESSCH         LCITY   LZIP  G1OFFERED  G8OFFERED  \
0           1   10000201667    Prattville  36067          0          1   
1           1   10000201876      Mt Meigs  36057          0          0   
2           1   10000201706    Birmingham  35206          0          1   
3           1   10000201670   Thomasville  36784          0          1   
4           1   10000200277    Birmingham  35220          0          1   
5           1   10000201705   Mount Meigs  36057          0          1   
6           1   10000500889   Albertville  35950          0          0   
7           1   10000501616   Albertville  35951          0          0   
8           1   10000500870   Albertville  35950          0          1   
9           1   10000500879   Albertville  35950          0          0   
10          1   10000500871   Albertville  35950          0          0   
11          1   10000502150   Albertville  35950          1          0   
12          1   10000600876  Guntersville  35976          1          0   
13          1   10000600872   Albertville  35951          0          1   
14          1   10000600887   Union Grove  35175          1          0   
15          1   10000601413       Douglas  35964          1          0   
16          1   10000601685         Grant  35747          1          0   
17          1   10000600193         Grant  35747          0          1   
18          1   10000602209   Albertville  35951          1          0   
19          1   10000601585  Guntersville  35976          0          0   
20          1   10000601434  Guntersville  35976          0          1   
21          1   10000600987  Guntersville  35976          0          0   
22          1   10000600883         Grant  35747          0          0   
23          1   10000600986  Guntersville  35976          0          1   
24          1   10000600880          Arab  35016          0          0   
25          1   10000600878       Douglas  35964          0          0   
26          1   10000600877       Douglas  35964          0          0   
27          1   10000601812       Douglas  35964          0          1   
28          1   10000701456        Hoover  35242          0          0   
29          1   10000702186        Hoover  35244          0          0   
...       ...           ...           ...    ...        ...        ...   
102371     78  780000200035   Saint Croix    840          0          0   
102372     78  780000200030   Saint Croix    851          0          0   
102373     78  780000200029   Saint Croix    850          1          0   
102374     78  780000200037   Saint Croix    850          0          1   
102375     78  780000200028   Saint Croix    820          1          0   
102376     78  780000200023   Saint Croix    820          1          0   
102377     78  780000200021   Saint Croix    820          1          0   
102378     78  780000200013   Saint Croix    850          0          0   
102379     78  780000200011   Saint Croix    840          1          0   
102380     78  780000200009   Saint Croix    820          0          1   
102381     78  780000200007   Saint Croix    850          0          0   
102382     78  780000200006   Saint Croix    840          1          0   
102383     78  780000200004   Saint Croix    851          0          0   
102384     78  780000200003   Saint Croix    840          0          1   
102385     78  780000200002   Saint Croix    820          1          0   
102386     78  780000200001   Saint Croix    820          1          0   
102387     78  780003000034  Saint Thomas    802          0          1   
102388     78  780003000033  Saint Thomas    804          0          1   
102389     78  780003000027  Saint Thomas    802          1          0   
102390     78  780003000026  Saint Thomas    802          1          0   
102391     78  780003000025  Saint Thomas    802          1          0   
102392     78  780003000024  Saint Thomas    802          1          0   
102393     7

In [118]:
all_data=pd.merge(school_dir, school_member, how='left', on='NCESSCH')
all_data

FIPST       NCESSCH         LCITY   LZIP  G1OFFERED  G8OFFERED  \
0           1   10000201667    Prattville  36067          0          1   
1           1   10000201876      Mt Meigs  36057          0          0   
2           1   10000201706    Birmingham  35206          0          1   
3           1   10000201670   Thomasville  36784          0          1   
4           1   10000200277    Birmingham  35220          0          1   
5           1   10000201705   Mount Meigs  36057          0          1   
6           1   10000500889   Albertville  35950          0          0   
7           1   10000501616   Albertville  35951          0          0   
8           1   10000500870   Albertville  35950          0          1   
9           1   10000500879   Albertville  35950          0          0   
10          1   10000500871   Albertville  35950          0          0   
11          1   10000502150   Albertville  35950          1          0   
12          1   10000600876  Guntersville  35976          1          0   
13          1   10000600872   Albertville  35951          0          1   
14          1   10000600887   Union Grove  35175          1          0   
15          1   10000601413       Douglas  35964          1          0   
16          1   10000601685         Grant  35747          1          0   
17          1   10000600193         Grant  35747          0          1   
18          1   10000602209   Albertville  35951          1          0   
19          1   10000601585  Guntersville  35976          0          0   
20          1   10000601434  Guntersville  35976          0          1   
21          1   10000600987  Guntersville  35976          0          0   
22          1   10000600883         Grant  35747          0          0   
23          1   10000600986  Guntersville  35976          0          1   
24          1   10000600880          Arab  35016          0          0   
25          1   10000600878       Douglas  35964          0          0   
26          1   10000600877       Douglas  35964          0          0   
27          1   10000601812       Douglas  35964          0          1   
28          1   10000701456        Hoover  35242          0          0   
29          1   10000702186        Hoover  35244          0          0   
...       ...           ...           ...    ...        ...        ...   
102371     78  780000200035   Saint Croix    840          0          0   
102372     78  780000200030   Saint Croix    851          0          0   
102373     78  780000200029   Saint Croix    850          1          0   
102374     78  780000200037   Saint Croix    850          0          1   
102375     78  780000200028   Saint Croix    820          1          0   
102376     78  780000200023   Saint Croix    820          1          0   
102377     78  780000200021   Saint Croix    820          1          0   
102378     78  780000200013   Saint Croix    850          0          0   
102379     78  780000200011   Saint Croix    840          1          0   
102380     78  780000200009   Saint Croix    820          0          1   
102381     78  780000200007   Saint Croix    850          0          0   
102382     78  780000200006   Saint Croix    840          1          0   
102383     78  780000200004   Saint Croix    851          0          0   
102384     78  780000200003   Saint Croix    840          0          1   
102385     78  780000200002   Saint Croix    820          1          0   
102386     78  780000200001   Saint Croix    820          1          0   
102387     78  780003000034  Saint Thomas    802          0          1   
102388     78  780003000033  Saint Thomas    804          0          1   
102389     78  780003000027  Saint Thomas    802          1          0   
102390     78  780003000026  Saint Thomas    802          1          0   
102391     78  780003000025  Saint Thomas    802          1          0   
102392     78  780003000024  Saint Thomas    802          1          0   
102393     7

In [107]:
all_data=pd.merge(school_dir, school_member, how='left', on='NCESSCH')
all_data=pd.merge(all_data, school_staff, how='left', on='NCESSCH')
all_data=pd.merge(all_data, grad_2015, how='left', on='NCESSCH') # grad_2015 only includes greenville

In [111]:
all_data[all_data['type_school']=='high_school']

FIPST       NCESSCH           LCITY   LZIP  G1OFFERED  G8OFFERED  \
0           1   10000201667      Prattville  36067          0          1   
2           1   10000201706      Birmingham  35206          0          1   
3           1   10000201670     Thomasville  36784          0          1   
4           1   10000200277      Birmingham  35220          0          1   
5           1   10000201705     Mount Meigs  36057          0          1   
10          1   10000500871     Albertville  35950          0          0   
13          1   10000600872     Albertville  35951          0          1   
19          1   10000601585    Guntersville  35976          0          0   
21          1   10000600987    Guntersville  35976          0          0   
22          1   10000600883           Grant  35747          0          0   
23          1   10000600986    Guntersville  35976          0          1   
25          1   10000600878         Douglas  35964          0          0   
28          1   10000701456          Hoover  35242          0          0   
35          1   10000700251          Hoover  35244          0          0   
43          1   10000702359          Hoover  35216          0          1   
47          1   10000800831         Madison  35758          0          0   
49          1   10000802198         Madison  35756          0          0   
56          1   10000901406       Talladega  35160          0          1   
62          1   10001102096           Leeds  35094          0          0   
63          1   10001200874            Boaz  35957          0          0   
68          1   10001301682      Trussville  35173          0          0   
75          1   10001502121      Montgomery  36117          0          1   
76          1   10001502126      Montgomery  36117          0          1   
78          1   10001802104      Birmingham  35203          0          1   
82          1   10003000003  Alexander City  35010          0          0   
91          1   10004401855      Birmingham  35212          0          1   
92          1   10006000007       Andalusia  36420          0          0   
95          1   10009001246        Anniston  36206          0          1   
100         1   10009000011        Anniston  36207          0          0   
104         1   10010000028            Arab  35016          0          0   
...       ...           ...             ...    ...        ...        ...   
102231     72  720003001872      RIO GRANDE    745          0          0   
102239     72  720003001803           LOIZA    772          0          0   
102242     72  720003001800       VEGA BAJA    693          0          0   
102246     72  720003001891    AGUAS BUENAS    703          0          0   
102250     72  720003001788       VEGA ALTA    692          0          0   
102256     72  720003001901           CIDRA    739          0          0   
102258     72  720003001782          UTUADO    641          0          1   
102268     72  720003001762          UTUADO    641          0          0   
102275     72  720003001753         FLORIDA    650          0          0   
102278     72  720003001939        VILLALBA    766          0          0   
102281     72  720003001943        SAN JUAN    928          0          0   
102295     72  720003001984       AGUADILLA    603          0          0   
102296     72  720003001724     BARCELONETA    617          0          0   
102298     72  720003001991        SAN JUAN    907          0          1   
102301     72  720003001997         COROZAL    783          0          0   
102309     72  720003001707         ARECIBO    614          0          1   
102310     72  720003002007           YAUCO    698          0          1   
102321     72  720003001695    QUEBRADILLAS    678          0          0   
102343     72  720003002062   SAN SEBASTIAN    685          0          0   
102345     72  720003001658        OROCOVIS    720          0          0   
102347     72  720003001657        OROCOVIS    720      

In [91]:
all_data[all_data['LZIP']==29650]

FIPST       NCESSCH  LCITY   LZIP  G1OFFERED  G8OFFERED  G12OFFERED  \
78344     45  450231001365  Greer  29650          1          0           0   
78370     45  450231000571  Greer  29650          0          0           1   
78385     45  450231000540  Greer  29650          1          0           0   
78388     45  450231000579  Greer  29650          0          0           1   
78428     45  450231000459  Greer  29650          0          1           0   

       NOGRADES                       SCH_NAME    G01      ...        \
78344         0         Buena Vista Elementary  170.0      ...         
78370         0                 Riverside High   -2.0      ...         
78385         0            Woodland Elementary  168.0      ...         
78388         0  J. Harley Bonds Career Center    0.0      ...         
78428         0               Riverside Middle   -2.0      ...         

       ALL_RATE_1516  MAM_RATE_1516  MAS_RATE_1516  MBL_RATE_1516  \
78344            NaN            NaN            NaN            NaN   
78370           91.0      55.506429           80.0           90.0   
78385            NaN            NaN            NaN            NaN   
78388            NaN            NaN            NaN            NaN   
78428            NaN            NaN            NaN            NaN   

       MHI_RATE_1516  MTR_RATE_1516  MWH_RATE_1516  CWD_RATE_1516  \
78344            NaN            NaN            NaN            NaN   
78370           80.0      58.618379           90.0           50.0   
78385            NaN            NaN            NaN            NaN   
78388            NaN            NaN            NaN            NaN   
78428            NaN            NaN            NaN            NaN   

       ECD_RATE_1516  LEP_RATE_1516  
78344            NaN            NaN  
78370           75.0           80.0  
78385            NaN            NaN  
78388            NaN            NaN  
78428            NaN            NaN  

[5 rows x 46 columns]